# Introduction

The solution is adapted from [here](https://www.bnlearn.com/research/sachs05/), and it was initially designed for the Sachs dataset.

This set contains simultaneous measurements of 11 proteins and lipids, derived from thousands of individual primary immune system cells. The data is continuous, representing molecule concentrations, so the standard approach is to assume that the concentrations follow a Gaussian distribution. However, a quick analysis on the set, as seen in the appropriate notebook, shows that the distributions are actually different from a normal distributions, most of the components being strongly skewed. The cause of this is that the values represent molecular concentrations, thus rendering very small positive values, clustering around zero. Also, the dependency relationships are not always linear, as can be observed in the relative graphs, as seen also in the notebook associated to the set.

After analyzing possible alternatives, such as Gaussian Bayesian Networks (GBNs, which are not appropriate for the given type of data), applying monotone transformations, or hybrid models where prior knowledge on the signalling pathways should be known (available here, but not in all real world situations), the article mentioned before proposes as feasible a two-step solution inspired by the paper associated to the dataset.

Sachs et al. decide to discretize the data and model it with a discrete BN, in order to handle skewness, with the disadvantage of losing ordering information. The article suggests discretizing the data in 3 bins, representing 3 levels of concentration (low - medium - high). Also, the discretization method should be information preserving, thus Hartemink is employed. Initially, in order to lose as little information as possible, the data is divided into a large number of levels (in the default case, 60). After this, the algorithm iterates over all pairs of adjacent intervals, computing their pairwise mutual information. I iteratively collapses each pair that yields the minimal information loss into a single interval, recomputing it after each such unification. Taking this mutual information, this type of discretization should better reflect the pre-existent dependency structures, when compared to other, simpler, methods, such as quantile or interval.

The second steps consists in averaging the structure learned in multiple graphs, obtaining a more robust final one. The method used for this is applying bootstrap resampling, such that a number (default, 500) of graphs is learned, and then decide on a final one with hill-climbing greedy search from each of the initial ones.

For all examples below, two methods are explored: one where Hartemink discretization is applied before the hill climbing greedy search, and another one where the search is performed on the initial dataset, since the synthetic sets have distributions less skewed than the real one.

# 0. Preliminaries

This section includes the required imports. The notebook can be run locally after following the README instructions in the root directory.

In [1]:
from data.csuite.csuite_datasets import *
from models.bnlearn.bnlearn_models import bnlearn_csuite, bnlearn_sachs

# 1. Observational set (Sachs)

The function loads the Sachs set as df and applies the steps described in the introduction.

The output will contain the f1 scores, as described in the evaluation notebook, and the estimated conditional probability distributions.

The hyperparameters from the initial examples were used, where the discretization starts with 60 bins and goes down to 3. The greedy search is performed using the posterior probability (BGe).

In [2]:
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_sachs(apply_hartemink_discretization=True)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_sachs(apply_hartemink_discretization=False)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.
{'adjacency_f1': 0.5454545617103577, 'orientation_f1': 0.27272728085517883}



~~~~~ WITHOUT DISCRETIZATION ~~~~~
{'adjacency_f1': 0.6086956262588501, 'orientation_f1': 0.2608695328235626}


# 2. Synthetic sets (csuite)

The function generates a newly and randomly populated df, based on the given graph. The df will contain a custom number of samples (currently set at 2,000).

The output will contain the f1 scores computed with the function available in the causica library, and the estimated conditional probability distributions.

The hyperparameters from the initial examples were again used: discretization starts with 60 bins and goes down to 3, and the greedy search is performed using the posterior probability (BGe).

In [2]:
samples_number = 2000

In [3]:
# ~~~~~~~~ LINGAUSS ~~~~~~~~
dataset = lingauss
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_0) = N(0.014 + 0.505*x_0, 0.745)



~~~~~ WITHOUT DISCRETIZATION ~~~~~

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_0) = N(0.014 + 0.505*x_0, 0.745)


In [4]:
# ~~~~~~~~ LINEXP ~~~~~~~~
dataset = linexp
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(0.015, 1.054)
[LinearGaussianCPD] P(x_1 | x_0) = N(-0.013 + 0.505*x_0, 0.724)



~~~~~ WITHOUT DISCRETIZATION ~~~~~

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(0.015, 1.054)
[LinearGaussianCPD] P(x_1 | x_0) = N(-0.013 + 0.505*x_0, 0.724)


In [5]:
# ~~~~~~~~ NONLINGAUSS ~~~~~~~~
dataset = nonlingauss
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_0) = N(1.438 + 0.076*x_0, 0.996)



~~~~~ WITHOUT DISCRETIZATION ~~~~~

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 1.0, 'orientation_f1': 1.0}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_0) = N(1.438 + 0.076*x_0, 0.996)


In [6]:
# ~~~~~~~~ NONLIN SIMPSON ~~~~~~~~
dataset = nonlin_simpson
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.888888955116272, 'orientation_f1': 0.888888955116272}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_0) = N(-0.092 + -0.711*x_0, 0.039)
[LinearGaussianCPD] P(x_2 | x_1, x_0) = N(-1.051 + 1.036*x_1 + 1.578*x_0, 0.458)
[LinearGaussianCPD] P(x_3 | x_0, x_2) = N(-0.302 + 0.003*x_0 + 0.426*x_2, 0.037)



~~~~~ WITHOUT DISCRETIZATION ~~~~~

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.800000011920929, 'orientation_f1': 0.6000000238418579}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0 | x_1) = N(-0.124 + -1.307*x_1, 0.071)
[LinearGaussianCPD] P(x_1) = N(-0.053, 0.546)
[LinearGa

In [7]:
# ~~~~~~~~ SYMPROD SIMPSON ~~~~~~~~
dataset = symprod_simpson
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.75, 'orientation_f1': 0.5}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0) = N(-0.055, 1.004)
[LinearGaussianCPD] P(x_1 | x_3, x_2, x_0) = N(0.016 + 1.026*x_3 + 0.007*x_2 + 0.742*x_0, 0.275)
[LinearGaussianCPD] P(x_2) = N(0.739, 0.874)
[LinearGaussianCPD] P(x_3 | x_0) = N(0.004 + 0.687*x_0, 0.156)



~~~~~ WITHOUT DISCRETIZATION ~~~~~

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.75, 'orientation_f1': 0.25}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0 | x_3, x_1) = N(-0.022 + 0.270*x_3 + 0.448*x_1, 0.166)
[LinearGaussianCPD] P(x_1 | x_3) = N(0.008 + 1.838*x_3, 0.411)
[LinearGaussianCPD] P(x_2 | x_0) = N(0.735 + -0.077*

In [8]:
# ~~~~~~~~ LARGE BACKDOOR ~~~~~~~~
dataset = large_backdoor
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.692307710647583, 'orientation_f1': 0.615384578704834}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0 | x_3) = N(0.711 + 0.694*x_3, 0.391)
[LinearGaussianCPD] P(x_1 | x_3, x_0) = N(-0.827 + 0.126*x_3 + 1.097*x_0, 0.069)
[LinearGaussianCPD] P(x_2 | x_1, x_3, x_0) = N(-0.356 + 0.110*x_1 + 0.058*x_3 + 0.629*x_0, 0.393)
[LinearGaussianCPD] P(x_3) = N(-1.084, 1.025)
[LinearGaussianCPD] P(x_4 | x_1, x_2) = N(-0.474 + 0.033*x_1 + 0.686*x_2, 0.394)
[LinearGaussianCPD] P(x_5 | x_1, x_2, x_3) = N(-0.497 + 0.061*x_1 + -0.028*x_2 + 0.570*x_3, 0.421)
[LinearGaussianCPD] P(x_6 | x_2, x_4) = N(-0.540 + -0.012*x_2 + 0.618*x_4, 0.409)
[LinearGaussian

In [9]:
# ~~~~~~~~ WEAK ARROWS ~~~~~~~~
dataset = weak_arrows
# With discretization
print("~~~~~ WITH DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=True,
    estimate_cpds=True
)

# Without discretization
print("\n\n\n~~~~~ WITHOUT DISCRETIZATION ~~~~~")
bnlearn_csuite(
    csuite_dataset=dataset,
    samples=samples_number,
    apply_hartemink_discretization=False,
    estimate_cpds=True
)

~~~~~ WITH DISCRETIZATION ~~~~~
Working on the 60th level of discretization.
Working on the 50th level of discretization.
Working on the 40th level of discretization.
Working on the 30th level of discretization.
Working on the 20th level of discretization.
Working on the 10th level of discretization.

~~~~~~~~~~ RESULTS ~~~~~~~~~~
{'adjacency_f1': 0.7058823704719543, 'orientation_f1': 0.1764705926179886}


~~~~~~~~~~ CPDs ~~~~~~~~~~
[LinearGaussianCPD] P(x_0 | x_5, x_1, x_2) = N(0.104 + 0.016*x_5 + 0.386*x_1 + -0.581*x_2, 0.347)
[LinearGaussianCPD] P(x_1 | x_5, x_3) = N(-0.283 + 0.039*x_5 + -0.762*x_3, 0.353)
[LinearGaussianCPD] P(x_2 | x_8, x_1) = N(-0.469 + 0.152*x_8 + -0.323*x_1, 0.644)
[LinearGaussianCPD] P(x_3 | x_8, x_7) = N(-0.267 + 0.044*x_8 + 0.620*x_7, 0.487)
[LinearGaussianCPD] P(x_4 | x_7, x_8, x_2) = N(-0.710 + -0.165*x_7 + 0.215*x_8 + 0.659*x_2, 0.365)
[LinearGaussianCPD] P(x_5 | x_3, x_7) = N(-0.952 + 0.175*x_3 + 0.751*x_7, 0.083)
[LinearGaussianCPD] P(x_6 | x_0, x_5, x_

# 3. Conclusions
Regarding the **Sachs dataset**, the adjacency f1 score is higher for the non-discretized solution, but the second one gives a marginally higher orientation f1 score. This may be explained by the fact that a gaussian network was estimated, instead of discrete one (this latter one is not implemented by the library that we used).

When evaluating the edges predicted by the solution, it can be observed that we were able to replicate the results from the article. Another common point is the significantly lower orientation score, since the network gives out scores close to 0.5 regarding the orientation (thus being unsure which node is the parent, and which one is the child in a binary relation).


-----


As for the **synthetic set**, we obtain perfect results with both methods, for the simple graphs, the discretization works better for the graphs that present non-linearities, and the two most complex graphs, large backdoor and weak arrows (that are similar in structure, with the latter graph containing a few extra edges compared to the former one), obtain significantly better results for both f1 scores without discretization, probably due to the fact that all information is preserved.

And looking at their CPDs, as expected, the simple models are easily estimated, where gaussian distributions are only lightly altered, but the complex ones also get good information regarding dependency, with a light imprecision when it comes to orientation.

Other scores were also tested, but the score from the original solution, bge, gives the best results.

Below, there is a comparison of the f1 scores for all datasets, first for undiscretized sets, and then for discretized ones. 

|                     | **Adjacency f1** | **Orientation f1** |
|---------------------|:----------------:|:------------------:|
|        **Lingauss** |       1.00       |        1.00        |
|          **Linexp** |       1.00       |        1.00        |
|     **Nonlingauss** |       1.00       |        1.00        |
|  **Nonlin Simpson** |       0.80       |        0.60        |
| **Symprod Simpson** |       0.75       |        0.25        |
|  **Large Backdoor** |       0.90       |        0.90        |
|     **Weak Arrows** |       0.83       |        0.76        |
|           **Sachs** |       0.61       |        0.26        |

Undiscretized

|                     | **Adjacency f1** | **Orientation f1** |
|---------------------|:----------------:|:------------------:|
|        **Lingauss** |       1.00       |        1.00        |
|          **Linexp** |       1.00       |        1.00        |
|     **Nonlingauss** |       1.00       |        1.00        |
|  **Nonlin Simpson** |       0.89       |        0.89        |
| **Symprod Simpson** |       0.75       |        0.50        |
|  **Large Backdoor** |       0.69       |        0.62        |
|     **Weak Arrows** |       0.71       |        0.18        |
|           **Sachs** |       0.55       |        0.27        |

Discretized